In [1]:
import pandas as pd
import numpy as np
import time
import gc

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [16]:
!gdown 1E7EKrW2wuCmH3ijw5KjT8loOH1lFvd_0
!unzip skillbox-recommender-system.zip -d recsys

Downloading...
From (original): https://drive.google.com/uc?id=1E7EKrW2wuCmH3ijw5KjT8loOH1lFvd_0
From (redirected): https://drive.google.com/uc?id=1E7EKrW2wuCmH3ijw5KjT8loOH1lFvd_0&confirm=t&uuid=30554d17-5f05-4a10-b0bb-dfbf39c752ad
To: /content/skillbox-recommender-system.zip
100% 163M/163M [00:01<00:00, 135MB/s]
Archive:  skillbox-recommender-system.zip
replace recsys/products.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace recsys/products.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace recsys/products.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: recsys/products.csv     
replace recsys/transactions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: recsys/transactions.csv  


**transactions.csv**
- order_id - уникальный идентификатор транзакции
- user_id - уникальный идентификатор покупателя
- order_number - порядковый номер транзакции в истории покупок данного пользователя
- order_dow - день недели транзакции
- order_hour_of_day - час совершения транзакции
- days_since_prior_order - количество дней с совершения предыдущей транзакции данным пользователем
- product_id - уникальный идентификатор товара
- add_to_cart_order - порядковый номер под которым данный товар был добавлен в корзину
- reordered - был ли товар "перезаказан"

**products.csv**
- product_id - уникальный идентификатор товара
- product_name - название товара
- aisle_id - уникальный идентификатор подкатегории
- department_id - уникальный идентификатор категории
- aisle - название подкатегории
- department - название категории

In [2]:
transactions = pd.read_csv('recsys/transactions.csv')
products = pd.read_csv('recsys/products.csv')

In [8]:
transactions.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,1,2,8,NaN,196,1.0,0.0
1,2539329,1,1,2,8,NaN,14084,2.0,0.0
2,2539329,1,1,2,8,NaN,12427,3.0,0.0
3,2539329,1,1,2,8,NaN,26088,4.0,0.0
4,2539329,1,1,2,8,NaN,26405,5.0,0.0


In [ ]:
products.head()

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,2,All-Seasons Salt,104,13,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry


# Top-n approach

In [30]:
purchase_top = transactions.groupby(['user_id', 'product_id'])['order_id'].count()
purchase_top = purchase_top.reset_index()
purchase_top = purchase_top.sort_values(['user_id', 'order_id'], ascending=[True, False]).reset_index(drop=True)

top_10_products = purchase_top.groupby('user_id').head(10)

In [42]:
d = {}
for uid in tqdm(top_10_products['user_id'].unique()):
    tmp = list(map(str, list(top_10_products[top_10_products['user_id']==uid]['product_id'].values)))
    d[uid] = ' '.join(tmp)

100%|██████████| 100000/100000 [04:15<00:00, 390.68it/s]


In [51]:
submit = pd.DataFrame(d.items(), columns=['user_id', 'product_id'])

In [52]:
submit.head()

,user_id,product_id
0,1,196 12427 10258 25133 13032 46149 13176 26088 ...
1,2,32792 47209 24852 1559 18523 19156 12000 16589...
2,3,39190 47766 21903 9387 17668 43961 1819 16797 ...
3,7,40852 37602 17638 21137 13198 31683 4920 30391...
4,13,27086 4210 27435 1689 33735 5652 25134 43086 4...


In [53]:
submit.to_csv('submission.csv', index=None)

# Attempt for CatBoost classification

In [7]:
data_small = pd.merge(transactions[transactions['user_id']<1000], products, 'inner', 'product_id')

del transactions
del products

gc.collect()

474

In [5]:
data_small.shape

(121428, 14)

In [6]:
data_small.nunique()

order_id                  12172
user_id                     480
order_number                 99
order_dow                     7
order_hour_of_day            24
days_since_prior_order       31
product_id                12727
add_to_cart_order            89
reordered                     2
product_name              12727
aisle_id                    134
department_id                21
aisle                       134
department                   21
dtype: int64

In [5]:
def nunique(l):
    return len(set(l))

# def count(l):
#     return len(l)

- *u_n_orders: число заказов п. А.*
- **u_n_transactions: число транзакций пользователя А.**  
- **u_unique_items: число уникальных товаров в истории п. А.**
- **u_order_size_mid: средний размер корзины п. А.**


- **i_n_popularity: число пользователей, купивших товар Б хотя бы раз.**
- i_n_orders_mid: среднее число заказов товара Б среди пользователей, купивших его хотя бы раз.
- i_days_delay_global_mid: медианная задержка (дней) между покупками товара Б среди всех пользователей.


- ui_n_chances: число заказов п. А после первой покупки товара Б (включительно).
- **ui_total_buy: число заказов товара Б пользователем А.**
- **ui_total_buy_ratio: = ui_total_buy / u_n_orders доля заказов п. А в которых есть товар Б.**
- ui_chance_buy_ratio: = ui_total_buy / u_n_chances доля заказов п. А, в которых есть товар Б, после первой покупки товара Б.
- **ui_avg_cart_pos: средняя позиция товара Б. в корзине пользователя А.**
- ui_days_delay_max: максимальная задержка (дней) между покупками товара Б пользователем А.
- ui_days_delay_mid: медианная задержка (дней) между покупками товара Б пользователем А.
- ui_days_passed: прогнозируемое число дней до следующего заказа пользователя А.
- ui_readyness_max: дней до следующего заказа п. А сверх своей максимальной задержки между покупками товара Б.
- ui_readyness_max_abs: модуль значения readyness. Большое значение сигнализирует о нетипичном отклонении в поведении пользователя относительно своего обычного поведения.
- ui_readyness_mid: аналогично ui_readyness_max для медианы.
- ui_readyness_mid_abs: аналогично ui_readyness_max_abs для медианы.
- ui_readyness_global_mid: задержка п. А в покупке товара Б сверх общей задержки среди всех пользователей покупающих этот товар.
- ui_readyness_global_mid_abs: модуль значения ui_readyness_global_mid. Большое значение сигнализирует о нетипичном отклонении в поведении пользователя относительно других пользователей.

## User features

In [ ]:
# Число товаров в конкретном заказе пользователя А
data_small['u_order_size'] = data_small.groupby(['user_id', 'order_number'])['product_id'].transform(len)

# средний размер корзины пользователя А
data_small['u_mean_order_size'] = data_small.groupby(['user_id'])['u_order_size'].transform(np.mean)

# число транзакций у пользователя А
data_small['u_transaction_num'] = data_small.groupby(['user_id'])['order_number'].transform(nunique)

# число уникальных товаров у пользователя А
data_small['u_num_unique_items'] = data_small.groupby(['user_id'])['product_id'].transform(nunique)

04.07.2024 08:09


## Item features

In [9]:
# сколько раз купили товар Б
data_small['i_num_purchases'] = data_small.groupby(['product_id'])['order_id'].transform(len)

# сколько пользователей покупали товар Б
data_small['i_unique_purchasers'] = data_small.groupby(['product_id'])['user_id'].transform(nunique)

# сколько раз пользователи в среднем покупает товар Б
data_small['i_mean_purchases'] = data_small['i_num_purchases'] / data_small['i_unique_purchasers']

# средний интервал между заказами товара Б
data_small['i_mean_interval_between_orders'] = data_small.groupby(['product_id'])['days_since_prior_order'].transform(np.mean)

## User-item features

In [28]:
# на каком месте в корзине пользователя А обычно лежит товар Б
data_small['ui_avg_item_pos'] = data_small.groupby(['user_id', 'product_id'])['add_to_cart_order'].transform(np.mean)

# сколько раз пользователь А купил товар Б
data_small['ui_num_purchases'] = data_small.groupby(['user_id', 'product_id'])['order_id'].transform(len)

# средний интервал между заказами товара А у пользователя Б
data_small['ui_mean_interval_between_purchases'] = data_small.groupby(['user_id', 'product_id'])['days_since_prior_order'].transform(np.mean)

# сколько дней пользователю А осталось до среднего интервала товара Б
data_small['ui_readyness_to_buy'] = data_small['days_since_prior_order'] - data_small['ui_mean_interval_between_purchases']

# в скольки заказах пользователя А есть товар Б
data_small['ui_num_transactions_with_item'] = data_small.groupby(['user_id', 'product_id'])['order_id'].transform(len)

# доля заказов пользователя А в которых есть товар Б
data_small['ui_item_purchase_rate'] = data_small['ui_num_transactions_with_item'] / data_small['u_transaction_num']

# средняя задержка пользователя А между покупками товара Б
data_small['ui_mean_purchase_delay'] = data_small.groupby(['user_id', 'product_id'])['days_since_prior_order'].transform(np.mean)

In [30]:
data_small[data_small['product_id']==13032][['user_id', 'order_id', 'days_since_prior_order', 'ui_mean_purchase_delay']]#.groupby(['user_id', 'product_id'])['order_id'].count()

,user_id,order_id,days_since_prior_order,ui_mean_purchase_delay
374225,1,2398795,15.0,21.666667
374226,1,550135,20.0,21.666667
374227,1,2550362,30.0,21.666667
374228,310,2450837,5.0,5.000000


In [14]:
data_small[(data_small['user_id']==1) & (data_small['product_id']==13032)]

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,u_transaction_num,u_num_unique_items,i_num_purchases,i_unique_purchasers,i_mean_purchases,i_mean_interval_between_orders,ui_avg_item_pos,ui_num_purchases,ui_mean_interval_between_purchases,ui_readyness_to_buy
374225,2398795,1,2,3,7,15.0,13032,6.0,0.0,Cinnamon Toast Crunch,...,10,18,4,2,2.0,17.5,6.333333,3,21.666667,-6.666667
374226,550135,1,7,1,9,20.0,13032,5.0,1.0,Cinnamon Toast Crunch,...,10,18,4,2,2.0,17.5,6.333333,3,21.666667,-1.666667
374227,2550362,1,10,4,8,30.0,13032,8.0,1.0,Cinnamon Toast Crunch,...,10,18,4,2,2.0,17.5,6.333333,3,21.666667,8.333333


In [ ]:
d1 = data_small[data_small['user_id'].isin([1, 2])][['user_id', 'order_id', 'order_number', 'product_id', 'u_order_size', 'u_mean_order_size',
                                                     'u_transaction_num', 'u_num_unique_items', 'i_unique_purchasers', 'i_num_purchases',
                                                     'i_mean_purchases', 'ui_avg_item_pos', 'ui_num_purchases', 'days_since_prior_order']]
# data_small.groupby(['product_id'])['order_number'].nunique()
i = 150
d1.iloc[i:i+10]

,user_id,order_id,order_number,product_id,u_order_size,u_mean_order_size,u_transaction_num,u_num_unique_items,i_unique_purchasers,i_num_purchases,i_mean_purchases,ui_avg_item_pos,ui_num_purchases,days_since_prior_order
1896930,2,1673511,5,17224,13,16.107692,14,102,6,24,4.000000,3.00,1,8.0
1903600,2,1673511,5,38656,13,16.107692,14,102,5,7,1.400000,4.00,1,8.0
1909140,2,1673511,5,21376,13,16.107692,14,102,9,41,4.555556,4.50,2,8.0
1909141,2,1718559,9,21376,26,16.107692,14,102,9,41,4.555556,4.50,2,8.0
1922039,2,1673511,5,48210,13,16.107692,14,102,2,3,1.500000,6.00,1,8.0
1924076,2,1673511,5,5907,13,16.107692,14,102,2,3,1.500000,7.00,1,8.0
1924570,2,1673511,5,14553,13,16.107692,14,102,6,6,1.000000,8.00,1,8.0
1927447,2,1673511,5,47553,13,16.107692,14,102,1,1,1.000000,9.00,1,8.0
1928135,2,1673511,5,22124,13,16.107692,14,102,10,36,3.600000,7.75,4,8.0
1928136,2,1718559,9,22124,26,16.107692,14,102,10,36,3.600000,7.75,4,8.0


In [ ]:
d1['user_id'].unique()

array([1, 2])

In [ ]:
data_small[(data_small['product_id']==196)]

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,u_order_size,u_mean_order_size,u_transaction_num,u_num_unique_items,i_num_purchases,i_unique_purchasers,i_mean_purchases,ui_avg_item_pos,ui_num_purchases,i_mean_interval_between_orders
0,2539329,1,1,2,8,NaN,196,1.0,0.0,Soda,...,5,6.254237,10,18,87,14,6.214286,1.4,10,10.170732
1,2398795,1,2,3,7,15.0,196,1.0,1.0,Soda,...,6,6.254237,10,18,87,14,6.214286,1.4,10,10.170732
2,473747,1,3,3,12,21.0,196,1.0,1.0,Soda,...,5,6.254237,10,18,87,14,6.214286,1.4,10,10.170732
3,2254736,1,4,4,7,29.0,196,1.0,1.0,Soda,...,5,6.254237,10,18,87,14,6.214286,1.4,10,10.170732
4,431534,1,5,4,15,28.0,196,1.0,1.0,Soda,...,8,6.254237,10,18,87,14,6.214286,1.4,10,10.170732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,2706088,290,28,6,10,7.0,196,16.0,1.0,Soda,...,27,33.966264,50,544,87,14,6.214286,12.0,2,10.170732
83,858092,420,4,1,19,30.0,196,2.0,0.0,Soda,...,7,7.467742,21,69,87,14,6.214286,2.0,2,10.170732
84,119314,420,5,3,19,16.0,196,2.0,1.0,Soda,...,3,7.467742,21,69,87,14,6.214286,2.0,2,10.170732
85,1665683,423,3,2,11,7.0,196,3.0,0.0,Soda,...,7,3.061224,41,30,87,14,6.214286,2.5,2,10.170732
